In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# (1)playsデータの「探索的データ分析」
# Exploratory data analysis for plays data
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/plays.csv')
plays.head(1000)

In [ ]:
#　空のカラムを作成
# Create an empty column
#　https://www.delftstack.com/ja/howto/python-pandas/how-to-create-an-empty-column-in-pandas-dataframe/
    
plays["incomplete_FLG"]= "NULL"
plays.head()

In [ ]:
# playsデータ
# 全PLAYデータのカラム絞ったもの
#「plays」のうち、カラムを限定したもの！
plays_lim = plays[["gameId","playId","playDescription","quarter","down","yardsToGo","absoluteYardlineNumber","playResult","incomplete_FLG"]]
plays_lim

In [ ]:
#filtering to view defensive players'complete play
# [df_offense_incomplete] = 特定の文字（incomplete）を含んだ列を抽出
# https://note.nkmk.me/python-pandas-index-columns-select/

df_offense_incomplete = pd.DataFrame(plays_lim.loc[plays_lim.playDescription.str.contains('incomplete','sack')])
df_offense_incomplete.head()

In [ ]:
# playsデータのうち、incompleteのものだけ抜き出し
# [df_offense_incomplete] = 特定の文字（incomplete）を含んだ列を抽出
#「df_offense_incomplete」のうち、カラムを3つ+[quarter,down,yardsToGo]に限定したもの！
df_offense_incomplete_X = df_offense_incomplete[["playId","quarter","down","yardsToGo","absoluteYardlineNumber","playResult"]]

#　新規列名を指定して追加
#　https://note.nkmk.me/python-pandas-assign-append/
df_offense_incomplete["incomplete_FLG"]= 1
df_offense_incomplete

In [ ]:
#filtering to view defensive players'complete play
# [df_offense_incomplete] = 特定の文字（incomplete）を含んだ列を抽出
# https://note.nkmk.me/python-pandas-index-columns-select/

df_offense_complete = pd.DataFrame(plays_lim.loc[~plays_lim.playDescription.str.contains('incomplete','sack')])
df_offense_complete

In [ ]:
# playsデータのうち、completeのものだけ抜き出し
# [df_offense_complete] = 特定の文字（incomplete）を含まない列を抽出
#「df_offense_complete」のうち、カラムを3つ+[quarter,down,yardsToGo]に限定したもの！
df_offense_complete_X = df_offense_complete[["playId","quarter","down","yardsToGo","absoluteYardlineNumber","playResult"]]

#　新規列名を指定して追加
#　https://note.nkmk.me/python-pandas-assign-append/
df_offense_complete["incomplete_FLG"]= 0
df_offense_complete

In [ ]:
#both data sets combined
# https://ai-inter1.com/pandas-dataframe_union/
df_plays=df_offense_incomplete.append(df_offense_complete, ignore_index=True)
df_plays.head(10)
# df_plays.sort_values(by='playId',ascending=True,inplace=True)

In [ ]:
# ascending=False を省略すると、昇順でのソート
# https://www.delftstack.com/ja/howto/python-pandas/how-to-sort-pandas-dataframe-by-one-column/
df_plays_order = df_plays.sort_values(by=['playId'],ignore_index=True)
df_plays_order

In [ ]:
# pandasでインデックスとは別の行に連番を振る
serial_num = pd.RangeIndex(start=1, stop=len(df_plays_order.index) + 1, step=1)
df_plays_order['No'] = serial_num

# 並び替えを行う
df_plays_order = df_plays_order.loc[:, ['No','playId','playDescription','quarter','down','yardsToGo','absoluteYardlineNumber','playResult','incomplete_FLG']]

df_plays_order

In [ ]:
# df_playsデータのうち、カラムを3つ+[quarter,down,yardsToGo]に限定したもの！
df_plays_all = df_plays_order[["No","quarter","down","yardsToGo","absoluteYardlineNumber","playResult","incomplete_FLG"]]
df_plays_all.head(100)

In [ ]:
#欠損値を含むデータを削除
#drop all NAN
df_plays_all.dropna()
df_plays_all=df_plays_all.drop(columns=df_plays_all.select_dtypes(include='object').columns)

In [ ]:
plt.scatter(df_plays_all['absoluteYardlineNumber'],df_plays_all['playResult'])

In [ ]:
# 線形回帰
#　linear_modelで説明変数を'down','yardsToGo'として、incomplete_FLGを予測
#　linear_model
X100 = df_plays_all[['quarter','down','yardsToGo','playResult']]
Y100 = df_plays_all['incomplete_FLG']
X100

In [ ]:
from sklearn.linear_model import LinearRegression
reg100 = LinearRegression()

In [ ]:
#Base Regression 70% split
X100_train, X100_test, Y100_train, Y100_test = train_test_split(X100,Y100,test_size = 0.3)

In [ ]:
#create a model and fit
reg100.fit(X100_train,Y100_train)

In [ ]:
#print the coefficients for each column and/or feature of the model

print(reg100.coef_)

In [ ]:
#predictons on the test data
Y100_pred = reg100.predict(X100_test)
Y100_pred
# print(np.round(Y100_pred).astype(int))

In [ ]:
#Actual output observed
Y100_test

In [ ]:
#modelA
#model accuracy measured by MSE
(np.mean((Y100_pred - Y100_test)**2))

In [ ]:
#modelA
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(Y100_test, Y100_pred)

In [ ]:
# 以下ランダムフォレスト

In [ ]:
#modelB　ランダムフォレスト
#ランダムフォレストで説明変数を'down','yardsToGo'として、incomplete_FLGを予測
##RandomForestRegressorModel using incomplete_FLG as dependent
# 
X100_forest = df_plays_all[['quarter','down','yardsToGo','playResult']]
Y100_forest = df_plays_all['incomplete_FLG']
X100_forest

In [ ]:
#modelB　ランダムフォレスト
#Test 70% Train30%
# from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

X100_forest_train,X100_forest_test,Y100_forest_train,Y100_forest_test = train_test_split(X100_forest,Y100_forest,test_size= 0.3)

In [ ]:
df_Y100_forest_test = pd.DataFrame(Y100_forest_test)
df_Y100_forest_test

In [ ]:
#modelB　ランダムフォレスト
#Importing RandomForestRegressor 
#　以下URLはダメ　∵RandomForestClassifier　×　https://qiita.com/nanairoGlasses/items/93d764f549943d42d5e6　　
#　こちらのURLはOK　∵RandomForestRegressor　〇　http://tekenuko.hatenablog.com/entry/2016/09/20/222453

# モデル構築、パラメータはデフォルト
from sklearn.ensemble import RandomForestRegressor

forest100 = RandomForestRegressor(random_state=1, n_estimators=300,max_leaf_nodes = 10,max_depth=12, max_features=None, min_samples_split = 0.3)
forest100.fit(X100_forest_train, Y100_forest_train)

In [ ]:
#modelB　ランダムフォレスト
#3割のテストデータで予測（x_forest1_test(テストデータ)）を代入
#　Substitute predictions with 30% test data

Y100_forest_pred = forest100.predict(X100_forest_test)
Y100_forest_pred

In [ ]:
df_Y100_forest_pred = pd.DataFrame(np.where(Y100_forest_pred >= 0.5, 1,0),Y100_forest_test.index,columns =["Y100_forest_pred"])
df_Y100_forest_pred

In [ ]:
# pandasでインデックスとは別の行に連番を振る
serial_num = pd.RangeIndex(start=1, stop=len(df_Y100_forest_pred.index) + 1, step=1)
df_Y100_forest_pred['No'] = serial_num

# 並び替えを行う
df_Y100_forest_pred = df_Y100_forest_pred.loc[:, ['No','Y100_forest_pred']]

df_Y100_forest_pred

In [ ]:
df_Y100_forest_test = pd.DataFrame(Y100_forest_test,columns =["incomplete_FLG"])
df_Y100_forest_test

In [ ]:
# pandasでインデックスとは別の行に連番を振る
serial_num = pd.RangeIndex(start=1, stop=len(df_Y100_forest_test.index) + 1, step=1)
df_Y100_forest_test['No'] = serial_num

# 並び替えを行う
df_Y100_forest_test = df_Y100_forest_test.loc[:, ['No','incomplete_FLG']]

df_Y100_forest_test

In [ ]:
df_forest = pd.merge(df_Y100_forest_pred, df_Y100_forest_test)
df_forest

In [ ]:
# csvファイルで出力

df_forest.to_csv('df_forest.csv',index=False)

In [ ]:
#modelB　ランダムフォレスト
# 学習の評価
# https://www.sejuku.net/blog/64374

forest100.score(X100_forest_test, Y100_forest_test)

Y100_forest_test

In [ ]:
#modelB
#check model performance/accuracy using Mean Squared Error　平均平方根誤差で正解率を確認 
#Y_pred（予測）とY_test（正解）の平方根誤差を見る
# 二乗平均平方根誤差　（RMSE：Root Mean Squared Error）＜評価＞誤差が小さいほうが精度は高い！

(np.mean((Y100_forest_pred - Y100_forest_test)**2))

In [ ]:
#modelB
# R2　（決定係数）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(Y100_forest_test, Y100_forest_pred)

Conclusion-1

I considered how the defense players would block offence player’s plays such as passes when the phase of the game was going on.
In my opinion, model B is better from the results of RMSE and R-squared.
From this result, I think it will be suggested how the phase of the game will develop to expand the playing field of defense players.
In order to motivate the defense players, it turned out that the current quarter and down, or how many yards left to get the next attack opportunity
 is important.


In [ ]:
#（2） playersデータの「探索的データ分析」
# Exploratory data analysis for players data

In [ ]:
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/players.csv')
players.head()

In [ ]:
#This is necessary to get rid of the heights with hyphens in them（身長部分のハイフンを除去）
players['height'] = players['height'].apply(lambda x: int(x[0])*12 + int(x[2]) if '-' in x else int(x))

In [ ]:
#生年月日から一時点の年齢を算出
#カラムに生まれ年'birthDate'を追加
#Converting birthDate to birthYear
players['birthDate'] = pd.to_datetime(players['birthDate']).dt.date
# Extracting the year
players['birthYear'] = pd.to_datetime(players['birthDate']).dt.year

In [ ]:
#オリジナルは2018年時点の年齢　→　2019年時点の年齢に変更
#カラムに年齢'age'を追加
#Converting from birthYear to Age at time of 2019
players['age'] = 2019 - players['birthYear']

players.head(10)

In [ ]:
players.shape

In [ ]:
#checking for any NA
players.isnull().sum()

In [ ]:
#Allows me to use ILOC to filter data
df = pd.DataFrame(players)
df.head(1303)

In [ ]:
#filtering to only view secondary defensive players
DB_pos = ['CB','SS','FS','DB']
df.loc[df['position'].isin(DB_pos)]
df = df.loc[df['position'].isin(DB_pos)]
df

In [ ]:
# #visualizing defensive backs broken down by college name
# plt.figure(figsize=(24,8))
# graph = sns.countplot(x='collegeName',data=df,palette='cividis',
#                       order=df.collegeName.value_counts().iloc[:15].index)

# for p in graph.patches:
#     height = p.get_height()
#     graph.text(p.get_x()+p.get_width()/2., height + 0.3,height ,ha="center")
    
# plt.show()

In [ ]:
# （3）weekデータの「探索的データ分析」
# Exploratory data analysis for week data

In [ ]:
# weekデータ
# week1 から week17 までのファイルを結合

In [ ]:
# weekデータ
# Pandasで複数のファイルを結合する方法【同じフォルダ内のcsvやexcelファイル等】
# https://life-freedom888.com/pandas-hukusuufile-concat/
# 同じディレクトリ内（フォルダ内）&同じファイル形式（csvなど）であれば、一括で結合可能（3つ以上でも問題なし）

import pandas as pd
import glob

In [ ]:
# weekデータ
# https://qiita.com/amowwee/items/e63b3610ea750f7dba1b
#　"week"で始まるファイル,ディレクトリのみを取得 ワイルドカードを指定
#  kaggleはinputの直下にフォルダが作れないので、一括は無理　← 一括でディレクトリ（フォルダ）内のcsvファイルを一括で読み込み

csv_files = glob.glob('../input/nfl-big-data-bowl-2021/week*')
list = []

In [ ]:
#weekデータ
####　　～～～～～～～～～～1回kaggle環境で作成したら、グレーアウト＜その1＞～～～～～～～～～～
for file in csv_files:
    list.append(pd.read_csv(file))

In [ ]:
#weekデータ
#pandasのconcatによって、リスト（つまりフォルダ内のすべてのcsv）を結合
#～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～
####　　～～～～～～～～～～1回kaggle環境で作成したら、グレーアウト＜その2＞～～～～～～～～～～
df_weekall = pd.concat(list)

In [ ]:
# weekデータ
# csvファイルで出力
#～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～
####　　～～～～～～～～～～1回kaggle環境で作成したら、グレーアウト＜その3＞ 以上～～～～～～

df_weekall.to_csv('weekall.csv',index=False)

In [ ]:
# weekデータ
#「df1」＝weekall(1~17の結合)のうち、カラムを５つ+[nflId,height,weight]に限定したもの！
df_weekall = pd.read_csv('./weekall.csv')
df1 = df_weekall[["s","a","dis","displayName","position","nflId"]]

In [ ]:
# weekデータ
df1 

In [ ]:
# weekデータ
# import weekall data
# =================kaggle 環境　ver================
# weekall = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/weekall.csv')
# =================================================

In [ ]:
# weekデータ
df1.isnull().sum()

In [ ]:
# weekデータ
#weekデータから不要なカラムを削除（以下の5カラムだけ残すs（速度＜yards/秒>）、a(加速度＜Sの2乗>、dis（前の時点からの移動距離）、displayName、position）
# weekall.drop(['dir','nflId','o','jerseyNumber','route','time','x','y','event','frameId','team','gameId','playId','playDirection'],axis=1)
# weekall = weekall.drop(['dir','nflId','o','jerseyNumber','route','time','x','y','event','frameId','team','gameId','playId','playDirection'],axis=1)
# weekall

In [ ]:
# weekデータ
#欠損値を含むデータを削除
#drop all NAN
df1.dropna()

In [ ]:
# weekデータ
# df1 = pd.DataFrame(week1)
# df1

In [ ]:
# weekデータ
#「df1_pos」＝ weekallのうちカラムを5つに絞り、かつ対象をDBだけに限定して、データを作成（98.6万件→33.1万件）
#filtering so only DB's are captured
DB_Pos = ['CB','SS','FS','DB']
df1.loc[df1['position'].isin(DB_Pos)]
df1_pos = df1.loc[df1['position'].isin(DB_Pos)]
df1_pos

In [ ]:
# weekデータ 「df2」=「df1_pos」＝ weekallのうちカラムを5つに絞り、かつ対象をDBだけに限定
df2 = pd.DataFrame(df1_pos)
df2

In [ ]:
# weekデータ 「df2」=「df1_pos」＝ weekallのうちカラムを5つに絞り、かつ対象をDBだけに限定
# The average acceleration and speed for Defensive Back during week 9
df2.groupby('displayName').mean('s')
df2 = df2.groupby('displayName').mean('s')
df2

In [ ]:
## Scatter Plot representing acceleration on 'y' axis and speed on 'x' axis
# ax1 = df2.plot.scatter(x='s', y='a', c='Green')
# ax2 = df.plot.scatter(x='height', y='weight', c='Red')
# ax3 = df.plot.scatter(x='height', y='age', c='Blue')

In [ ]:
#　https://medium-s.jp/kaggle-house-prices/
#sとaの外れ値を除外する
df2 = df2.drop(df2[(df2['s']>4) & (df2['a']<2.2)].index)
df2 = df2.drop(df2[(df2['s']<2) & (df2['a']>2.2)].index)

# 身長データなしの外れ値を除外する
df = df.drop(df[(df['height']<62)& (df['weight']>170)].index)

In [ ]:
# # Scatter Plot representing acceleration on 'y' axis and speed on 'x' axis
# ax1 = df2.plot.scatter(x='s', y='a', c='Green')

In [ ]:
# # Scatter Plot representing acceleration on 'y' axis and speed on 'x' axis
# ax1 = df.plot.scatter(x='height', y='weight', c='Red')

In [ ]:
display(df,df2)

In [ ]:
#changing index to be able to merge 
df2.reset_index(inplace = True)
df2

In [ ]:
df2

In [ ]:
#both data sets combined
dfcombine = pd.merge(df,df2)
dfcombine

In [ ]:
#AVG Weight is 199.253
#AVG Age is 26.682540
dfcombine.describe()

In [ ]:
#Modeling Portion
#Code implemented will include linear regressions with weight and speed as independent variables and acceleration as dependent variable. 


In [ ]:
#model1
#(1)-1.線形回帰で説明変数を身長・体重・加速度a・速度sとして、年齢AGEを予測

x1 = dfcombine[['height','weight','a','s']]
y1 = dfcombine['age']
x1

In [ ]:
from sklearn.linear_model import LinearRegression
reg1 = LinearRegression()

In [ ]:
#Base Regression 70% split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1,test_size = 0.3)

In [ ]:
#create a model and fit
reg1.fit(x1_train,y1_train)

In [ ]:
#print the coefficients for each column and/or feature of the model

print(reg1.coef_)

In [ ]:
#predictons on the test data
y1_pred = reg1.predict(x1_test)
y1_pred

In [ ]:
#Actual output observed
y1_test

In [ ]:
#model1
#model accuracy measured by MSE
(np.mean((y1_pred - y1_test)**2))

In [ ]:
#model1
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y1_test, y1_pred)

In [ ]:
#model2
#(1)-2.線形回帰で説明変数を身長・加速度a・年齢AGE・速度sとして、体重を予測
#Linear Regression looking at weight as the dependent variable

x2 = dfcombine[['height','a','s','age']]
y2 = dfcombine['weight']
x2

In [ ]:
reg2 = LinearRegression()

In [ ]:
#Split of 70/30 between train and test

x2_train, x2_test, y2_train, y2_test = train_test_split(x2,y2,test_size = 0.3)

In [ ]:
#train using training data

reg2.fit(x2_train,y2_train)

In [ ]:
#display coefficients

print(reg2.coef_)

In [ ]:
#print y test data predictions
y2_pred = reg2.predict(x2_test)
y2_pred                    

In [ ]:
y2_test

In [ ]:
#model2
#Checking accuracy with MSE
mean_squared_error(y2_test, y2_pred)

In [ ]:
#model2
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y2_test, y2_pred)

In [ ]:
#model3
#(1)-3.線形回帰で説明変数を体重・加速度a・年齢AGE・速度sとして、身長を予測
#Linear Regression looking at weight as the dependent variable

x3 = dfcombine[['weight','a','s','age']]
y3 = dfcombine['height']
x3

In [ ]:
reg3 = LinearRegression()

In [ ]:
#Split of 70/30 between train and test

x3_train, x3_test, y3_train, y3_test = train_test_split(x3,y3,test_size = 0.3)

In [ ]:
#train using training data

reg3.fit(x3_train,y3_train)

In [ ]:
#display coefficients

print(reg3.coef_)

In [ ]:
#print y test data predictions
y3_pred = reg3.predict(x3_test)
y3_pred        

In [ ]:
y3_test

In [ ]:
#model3
#Checking accuracy with MSE
mean_squared_error(y3_test, y3_pred)

In [ ]:
#model3
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y3_test, y3_pred)

In [ ]:
#model4
#(1)-4.線形回帰で説明変数を体重・身長・年齢AGE・速度sとして、加速度aを予測
#Linear Regression looking at weight as the dependent variable

x4 = dfcombine[['weight','height','s','age']]
y4 = dfcombine['a']
x4

In [ ]:
reg4 = LinearRegression()

In [ ]:
#Split of 70/30 between train and test

x4_train, x4_test, y4_train, y4_test = train_test_split(x4,y4,test_size = 0.3)

In [ ]:
#train using training data

reg4.fit(x4_train,y4_train)

In [ ]:
#display coefficients

print(reg4.coef_)

In [ ]:
#print y test data predictions
y4_pred = reg4.predict(x4_test)
y4_pred     

In [ ]:
y4_test

In [ ]:
#model4
#Checking accuracy with MSE
mean_squared_error(y4_test, y4_pred)

In [ ]:
#model4
# 決定係数（R2）モデルの当てはまりの良さを示す指標

#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y4_test, y4_pred)

In [ ]:
#model5
#(1)-5.線形回帰で説明変数を体重・身長・年齢AGE・加速度aとして、速度sを予測
#Linear Regression looking at weight as the dependent variable

x5 = dfcombine[['weight','height','a','age']]
y5 = dfcombine['s']
x5

In [ ]:
reg5 = LinearRegression()

In [ ]:
#Split of 70/30 between train and test

x5_train, x5_test, y5_train, y5_test = train_test_split(x5,y5,test_size = 0.3)

In [ ]:
#train using training data

reg5.fit(x5_train,y5_train)

In [ ]:
#display coefficients

print(reg5.coef_)

In [ ]:
#print y test data predictions
y5_pred = reg5.predict(x5_test)
y5_pred     

In [ ]:
y5_test

In [ ]:
#model5
#Checking accuracy with MSE
mean_squared_error(y5_test, y5_pred)

In [ ]:
#model5
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y5_test, y5_pred)

In [ ]:
#model6
#(2)-1.ランダムフォレスト
#ランダムフォレストで説明変数を身長・体重・速度s・年齢AGEとして、加速度aを予測
##RandomForestRegressorModel using s as dependent
x_forest1 = dfcombine[['height','weight','s','age']]
y_forest1 = dfcombine['a']

In [ ]:
#Test 70% Train30%
from sklearn.model_selection import train_test_split

x_forest1_train, x_forest1_test, y_forest1_train, y_forest1_test = train_test_split(x_forest1,y_forest1,test_size= 0.3)

In [ ]:
#Importing RandomForestRegressor 
#　以下URLはダメ　∵RandomForestClassifier　×　https://qiita.com/nanairoGlasses/items/93d764f549943d42d5e6　　
#　こちらのURLはOK　∵RandomForestRegressor　〇　http://tekenuko.hatenablog.com/entry/2016/09/20/222453

# モデル構築、パラメータはデフォルト
from sklearn.ensemble import RandomForestRegressor

forest1 = RandomForestRegressor(random_state=1, n_estimators=300,max_leaf_nodes = 10,max_depth=12, max_features=None, min_samples_split = 0.3)
forest1.fit(x_forest1_train, y_forest1_train)

In [ ]:
#3割のテストデータで予測（x_forest1_test(テストデータ)）を代入
#　Substitute predictions with 30% test data

y_forest1_pred = forest1.predict(x_forest1_test)
y_forest1_pred

In [ ]:
# 学習の評価
# https://www.sejuku.net/blog/64374

forest1.score(x_forest1_test, y_forest1_test)

In [ ]:
y_forest1_test

In [ ]:
#model6
#check model performance/accuracy using Mean Squared Error　平均平方根誤差で正解率を確認 
#Y_pred（予測）とY_test（正解）の平方根誤差を見る
# 二乗平均平方根誤差　（RMSE：Root Mean Squared Error）＜評価＞誤差が小さいほうが精度は高い！

(np.mean((y_forest1_pred - y_forest1_test)**2))

In [ ]:
#model6
# R2　（決定係数）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y_forest1_test, y_forest1_pred)

In [ ]:
#model7
#(2)-2.ランダムフォレスト
#ランダムフォレストで説明変数を体重・加速度a・速度s・年齢AGEとして、身長を予測
##RandomForestRegressorModel using height as dependent
x_forest2 = dfcombine[['age','weight','a','s']]
y_forest2 = dfcombine['height']

In [ ]:
#Test 70% Train30%
from sklearn.model_selection import train_test_split

x_forest2_train, x_forest2_test, y_forest2_train, y_forest2_test = train_test_split(x_forest2,y_forest2,test_size= 0.3)

In [ ]:
#Importing RandomForestRegressor 
#　以下URLはダメ　∵RandomForestClassifier　×　https://qiita.com/nanairoGlasses/items/93d764f549943d42d5e6　　
#　こちらのURLはOK　∵RandomForestRegressor　〇　http://tekenuko.hatenablog.com/entry/2016/09/20/222453

# モデル構築、パラメータはデフォルト
from sklearn.ensemble import RandomForestRegressor

forest2 = RandomForestRegressor(random_state=1, n_estimators=300,max_leaf_nodes = 10,max_depth=12, max_features=None, min_samples_split = 0.3)
forest2.fit(x_forest2_train, y_forest2_train)

In [ ]:
#3割のテストデータで予測（X1_test(テストデータ)）を代入
#　Substitute predictions with 30% test data

y_forest2_pred = forest2.predict(x_forest2_test)
y_forest2_pred

In [ ]:
# 学習の評価
# https://www.sejuku.net/blog/64374

forest2.score(x_forest2_test, y_forest2_test)

In [ ]:
y_forest2_test

In [ ]:
#model7
#check model performance/accuracy using Mean Squared Error　平均平方根誤差で正解率を確認 
#Y_pred（予測）とY_test（正解）の平方根誤差を見る
# 二乗平均平方根誤差　（RMSE：Root Mean Squared Error）＜評価＞誤差が小さいほうが精度は高い！

(np.mean((y_forest2_pred - y_forest2_test)**2))

In [ ]:
#model7
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y_forest2_test, y_forest2_pred)

In [ ]:
# (3)頻出選手限定版
#Frequent player limited edition
# 以下は、頻出上位選手に限定して訓練する

In [ ]:
# リストのカラムの中から、名前の最頻値を上位から抜き出す
# https://note.nkmk.me/python-pandas-value-counts/
#　value_counts()　　ユニークな要素の値のリストに加えて、・ユニークな要素の頻度（出現回数）が必要な時　

#　pandas のデフォルトの表示から表示行を増やす
#　https://life-freedom888.com/pandas-shouryaku-none/
pd.set_option('display.max_rows',1400)

# weekデータ
#「df1」＝weekall(1~17の結合)のうち、カラムを５つ+[nflId,height,weight]に限定したもの！のなかから、頻出選手を抜き出す

df1_vc = df1['displayName'].value_counts()
print(df1_vc)
print(type(df1_vc))

In [ ]:
#filtering in the top 20% of players　that often appear in "week data"
#　上位20％の選手程度に絞る

top_20per = ["Michael Thomas","Ben Roethlisberger","JuJu Smith-Schuster","Malcolm Jenkins","Aaron Rodgers","Antonio Brown","Steven Nelson","Andrew Luck","Davante Adams","Adrian Amos","Matt Ryan","Kyle Fuller","Jamal Adams","Jessie Bates","Stephon Gilmore","Avery Williamson","Eli Manning","Devin McCourty","Tyrann Mathieu","Jared Goff","Desmond Trufant","Zach Ertz","Eric Weddle","Danny Trevathan","Kendall Fuller","Patrick Mahomes","Kirk Cousins","Robert Woods","Mike Evans","Marcus Williams","Adam Thielen","William Jackson","Josh Norman","Dak Prescott","Nelson Agholor","Matthew Stafford","Travis Kelce","Damarious Randall","DeAndre Hopkins","Sterling Shepard","Deshaun Watson","Jarvis Landry","Kyle Van No","Mason Foster","Ron Parker","Brandin Cooks","Tom Brady","Janoris Jenkins","Derek Carr","Byron Jones","Nigel Bradham","Case Keenum","Kareem Jackson","Prince Amukamara","Tramon Williams","Jeff Heath","Jamie Collins","Eddie Jackson","Tyreek Hill","Morris Claiborne","D.J.Swearinger","Kevin Byard","Damontae Kazee","Kenny Golladay","Terrell Edmunds","Sean Davis","Patrick Chung","Kyle Rudolph","Blake Martinez","Curtis Riley","Derwin James","Patrick Peterson","Jarrad Davis","B.W.Webb","James Bradberry","Marshon Lattimore","Jahleel Addae","Antoine Bethea","Christian McCaffrey","John Johnson","Roquan Smith","Jimmy Graham","Ha Ha Clinton-Di","Casey Hayward","Quandre Diggs","Jaylon Smith"]
dfcombine.loc[dfcombine['displayName'].isin(top_20per)]
df_top20per = dfcombine.loc[dfcombine['displayName'].isin(top_20per)]
df_top20per

In [ ]:
#resetting index so I can merge
df_top20per.reset_index(inplace = True)
df_top20per

In [ ]:
#df_top20per 上位選手限定版
df_top20per.describe()

In [ ]:
#　（4）上位選手限定　~上位選手の年齢と速度の関係
#　Correlation between age and speed

plt.scatter(df_top20per['age'],df_top20per['s'])

In [ ]:
#　（4）上位選手限定　~上位選手の体重と速度
#　Correlation between weight and speed

plt.scatter(df_top20per['weight'],df_top20per['s'])

In [ ]:
#　（4）上位選手限定　~上位選手の加速度と速度
#　Correlation between a and s

plt.scatter(df_top20per['a'],df_top20per['s'])

In [ ]:
#model8
#線形回帰
# 　（4）-1　上位選手限定, 説明変数を身長・体重・速度s・年齢AGEとして、加速度aを予測
#Linear Regression looking at a as the dependent variable

x01 = df_top20per[['height','weight','age','a']]
y01 = df_top20per['s']
x01

In [ ]:
#model8
#線形回帰
# 　（4）-1　上位選手限定,説明変数を身長・体重・速度s・年齢AGEとして、加速度aを予測
#Linear Regression looking at a as the dependent variable

from sklearn.linear_model import LinearRegression
reg01 = LinearRegression()

In [ ]:
#model8
#線形回帰
# 　（4）-1　上位選手限定,説明変数を身長・体重・速度s・年齢AGEとして、加速度aを予測
#Linear Regression looking at a as the dependent variable
#データを分割
#Split the data into 70% train and 30% test

x01_train, x01_test, y01_train,y01_test = train_test_split(x01,y01,test_size = 0.3)

In [ ]:
#学習
##train the model with training data

reg01.fit(x01_train,y01_train)

In [ ]:
#係数を表示
#print coefficients/weights for each feature/column of our model
print(reg01.coef_)

In [ ]:
#切片の算出
reg01.intercept_

In [ ]:
#テストデータを回帰式に代入して、予測
#print predicitions on test data for y
y01_pred = reg01.predict(x01_test)
y01_pred

In [ ]:
#print the actual values
y01_test

In [ ]:
#model8
#MSE and SKlearn.metrics
from sklearn.metrics import mean_squared_error
mean_squared_error(y01_test, y01_pred)

In [ ]:
#model8
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y01_test, y01_pred)

In [ ]:
#ここから、ランダムフォレスト　

In [ ]:
#model9
#ランダムフォレスト　
# 　（4）-1　上位選手限定,説明変数を身長・体重・速度s・年齢AGEとして、加速度aを予測
#RandomForest looking at a as the dependent variable

x11 = df_top20per[['height','weight','age','s']]
y11 = df_top20per['a']
x11

In [ ]:
# ランダムフォレスト　
#（4）-2　上位選手限定,説明変数を身長・体重・速度s・年齢AGEとして、加速度aを予測
#データを分割
#Split the data into 70% train and 30% test

x11_forest_train, x11_forest_test, y11_forest_train,y11_forest_test = train_test_split(x11,y11,test_size = 0.3)

In [ ]:
#（4)-2
#Importing RandomForestRegressor
#　以下URLはダメ　∵RandomForestClassifier　×　https://qiita.com/nanairoGlasses/items/93d764f549943d42d5e6　　
#　こちらのURLはOK　∵RandomForestRegressor　〇　http://tekenuko.hatenablog.com/entry/2016/09/20/222453

# モデル構築、パラメータはデフォルト
from sklearn.ensemble import RandomForestRegressor

forest11 = RandomForestRegressor(random_state=1, n_estimators=300,max_leaf_nodes = 10,max_depth=12, max_features=None, min_samples_split = 0.2)
forest11.fit(x11_forest_train, y11_forest_train)

In [ ]:
#3割のテストデータで予測（x11_forest_test(テストデータ)）を代入
#print predicitions on test data for y

y11_forest_pred = forest11.predict(x11_forest_test)
y11_forest_pred

In [ ]:
# 学習の評価
# https://www.sejuku.net/blog/64374

forest11.score(x11_forest_test, y11_forest_test)

In [ ]:
y11_forest_test

In [ ]:
#model9
#二乗平均平方根誤差で正解率を確認 
#Y_pred（予測）とY_test（正解）の平方根誤差を見る
# 二乗平均平方根誤差　（RMSE：Root Mean Squared Error）＜評価＞誤差が小さいほうが精度は高い！

(np.mean((y11_forest_pred - y11_forest_test)**2))

In [ ]:
#model9
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y11_forest_test, y11_forest_pred)

In [ ]:
# (4)-3 パラメータチューニングの実践
# https://qiita.com/nanairoGlasses/items/93d764f549943d42d5e6
# http://tekenuko.hatenablog.com/entry/2016/09/20/222453

In [ ]:
# データの準備および分割
#Split the data into 70% train and 30% test

x12_forest_train, x12_forest_test, y12_forest_train,y12_forest_test = train_test_split(x11,y11,test_size = 0.3)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# グリッドサーチ(パラメータ候補指定)用のパラメータ1~300までの10刻み
paramG = {'n_estimators':[1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
                         ,110, 120, 130, 140, 150, 160, 170, 180, 190, 200
                         ,210, 220, 230, 240, 250, 260, 270, 280, 290, 300]}

In [ ]:
# ランダムサーチ(パラメータ範囲指定)用のパラメータ 1~300
paramR = {'n_estimators':np.arange(300)}

In [ ]:
# モデル生成。上から順に、通常のランダムフォレスト、グリッドサーチ・ランダムフォレスト、
# ランダムサーチ・ランダムフォレスト。

RFC_raw  = RandomForestRegressor(random_state=0)
RFC_grid = GridSearchCV(estimator=RandomForestRegressor(random_state=0), param_grid=paramG, \
                        scoring='r2', cv=3)
RFC_rand = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=0), param_distributions=paramR, \
                        scoring='r2', cv=3)

In [ ]:
 print(pd.__version__)

In [ ]:
# 各モデルに学習を行わせる。
from sklearn.ensemble import RandomForestRegressor
#「.as_matrix().ravel()」を削除　∵エラーが出る
# RFC_raw.fit (x01_train, y01_train.as_matrix().ravel())
# RFC_grid.fit(x01_train, y01_train.as_matrix().ravel())
# RFC_rand.fit(x01_train, y01_train.as_matrix().ravel())

RFC_raw.fit(x12_forest_train, y12_forest_train)
RFC_grid.fit(x12_forest_train, y12_forest_train)
RFC_rand.fit(x12_forest_train, y12_forest_train)

In [ ]:
# 各モデルの n_estimators パラメータの確認。
#print('通常のランダムフォレストモデルにおける n_estimators         :  %d'  %RFC_raw.n_estimators)

print('通常のランダムフォレストモデルにおけるn_estimators:',RFC_raw.n_estimators)
print('グリッドサーチ・ランダムフォレストモデルにおけるn_estimators:',RFC_grid.best_estimator_.n_estimators)
print('ランダムサーチ・ランダムフォレストモデルにおけるn_estimators:',RFC_rand.best_estimator_.n_estimators)

In [ ]:
# # 各モデルの予測値の確認。
# #print('通常のランダムフォレストモデルにおける n_estimators         :  %d'  %RFC_raw.n_estimators)

# r2_score(y01_test, y01_pred)
# print('通常のランダムフォレストモデルにおける予測値:',r2score(y01_test,RFC_raw.predict(x01_test)))
# print('グリッドサーチ・ランダムフォレストモデルにおける予測値:',r2score(y01_test,RFC_grid.predict(x01_test)))
# print('ランダムサーチ・ランダムフォレストモデルにおける予測値:',r2score(y01_test,RFC_rand.predict(x01_test)))

In [ ]:
#model10
# (4)-3-1 グリッドサーチ版（モデルはランダムフォレスト）
from sklearn.ensemble import RandomForestRegressor

#学習データを使って学習
#下の　n_estimators=「」の数字の部分を手入力
# 「, min_sample_split = 0.2」は削除　∵エラーが出る

forest2_g = RandomForestRegressor(random_state=1, n_estimators=20,max_leaf_nodes=10, max_depth=12, max_features=None)
forest2_g.fit(x12_forest_train, y12_forest_train)

In [ ]:
#3割のテストデータで予測（X12_forest_test(テストデータ)）を代入
#print predicitions on test data for y

y12_g_pred = forest2_g.predict(x12_forest_test)
y12_g_pred

In [ ]:
# 学習の評価
# https://www.sejuku.net/blog/64374

forest2_g.score(x12_forest_test, y12_forest_test)

In [ ]:
y12_forest_test

In [ ]:
#model10
#check model performance/accuracy using Mean Squared Error　平均平方根誤差で正解率を確認 
#Y_pred（予測）とY_test（正解）の平方根誤差を見る
# 二乗平均平方根誤差　（RMSE：Root Mean Squared Error）＜評価＞誤差が小さいほうが精度は高い！

(np.mean((y12_g_pred - y12_forest_test)**2))

In [ ]:
#model10
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y12_forest_test, y12_g_pred)

In [ ]:
#model11
# (4)-3-2 ランダムサーチ版（モデルはランダムフォレスト）

from sklearn.ensemble import RandomForestRegressor

#学習データを使って学習
#下の　n_estimators=「」の数字の部分を手入力
#　「, min_sample_split = 0.2」を削除　∵エラーが出る
forest2_r = RandomForestRegressor(random_state=1, n_estimators=8,max_leaf_nodes=10, max_depth=12, max_features=None)
forest2_r.fit(x12_forest_train, y12_forest_train)

In [ ]:
#3割のテストデータで予測（X12_forest_test(テストデータ)）を代入
#print predicitions on test data for y

y12_r_pred = forest2_r.predict(x12_forest_test)
y12_r_pred

In [ ]:
# 学習の評価
# https://www.sejuku.net/blog/64374

forest2_r.score(x12_forest_test, y12_forest_test)

In [ ]:
#model11
#check model performance/accuracy using Mean Squared Error function
#　平均二乗誤差（MSE: Mean Squared Error） ＜評価＞誤差が小さいほうが精度は高い！
from sklearn.metrics import mean_squared_error
mean_squared_error(y12_forest_test, y12_r_pred)

In [ ]:
#model11
# 決定係数（R2）モデルの当てはまりの良さを示す指標
#　＜評価＞0~1　当てはまりの悪い時はマイナスも

from sklearn.metrics import r2_score
r2_score(y12_forest_test, y12_r_pred)

Conclusion-2

I analyzed all NFL defensive players. I analyzed which of the player's individual attributes and abilities are important to play. Specifically, I analyzed age, weight, height, speed, and acceleration. From my analysis, I consider that acceleration and weight are relatively useful variables, while others are less useful.

In my opinion, model10 is the best, based on the results of RMSE and R-squared.
In model10, we've narrowed down the top 20% of NFL defensive players that frequently appear in “week”.data, that is, the NFL's representative defensive players.

Speaking only of the NFL's representative defensive players, as you can see from model8 to model11, I think that the acceleration of defensive players may be particularly important for defense.
